## Notebook for exploratory analysis of _Cai Y et al 2020_ and _Cai Y et al 2022_ scRNA-Seq data using `scVI`

- **Developed by**: Carlos Talavera-López Ph.D

- **Modified by**: Mairi McClean
- **Institute of Computational Biology - Computational Health Centre - Helmholtz Munich**
- v221101; modified v230314

### Load required modules

In [1]:
import sys
import scvi
import torch
import anndata
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns

import numpy as np
import pandas as pd
import scanpy as sc
import numpy.random as random


from umap import UMAP
import warnings; warnings.simplefilter('ignore')

Global seed set to 0
/Users/mairi.mcclean/mambaforge/envs/scvi_local/lib/python3.10/site-packages/flax/struct.py:133: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/mairi.mcclean/mambaforge/envs/scvi_local/lib/python3.10/site-packages/flax/struct.py:133: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)
/Users/mairi.mcclean/mambaforge/envs/scvi_local/lib/python3.10/site-packages/flax/struct.py:133: FutureWarning: jax.tree_util.register_keypaths is deprecated, and will be removed in a future release. Please use `register_pytree_with_keys()` instead.
  jax.tree_util.register_keypaths(data_clz, keypaths)


In [2]:
# Should this be put on the server?
save_path = "/Volumes/Lacie/data_lake/Mairi_example/notebook_output/paths/"

In [3]:
%matplotlib inline
matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42

In [4]:
torch.cuda.is_available()

False

In [5]:
sc.settings.verbosity = 3
sc.logging.print_versions()
sc.settings.set_figure_params(dpi = 180, color_map = 'magma_r', dpi_save = 300, vector_friendly = True, format = 'svg')

-----
anndata     0.8.0
scanpy      1.9.3
-----
PIL                 9.4.0
absl                NA
appnope             0.1.3
asttokens           NA
attr                22.2.0
backcall            0.2.0
beta_ufunc          NA
binom_ufunc         NA
brotli              NA
certifi             2022.12.07
cffi                1.15.1
charset_normalizer  2.1.1
chex                0.1.6
colorama            0.4.6
comm                0.1.2
contextlib2         NA
cycler              0.10.0
cython_runtime      NA
dateutil            2.8.2
debugpy             1.6.6
decorator           5.1.1
docrep              0.3.2
executing           1.2.0
flax                0.5.0
fsspec              2023.3.0
h5py                3.8.0
hypergeom_ufunc     NA
idna                3.4
igraph              0.10.4
invgauss_ufunc      NA
ipykernel           6.21.2
ipywidgets          8.0.4
jax                 0.4.6
jaxlib              0.4.6
jedi                0.18.2
joblib              1.2.0
kiwisolver          1.4.4
leide

### Read in datasets

- Read in _Cai Y et al 2020_

In [6]:
# caiy2020 = sc.read_h5ad('/lustre/groups/talaveralopez/datasets/tuberculosis/rna-seq/mairi_data_lake/processed_files/abridged_qc/human/Cai2020_scRNA_PBMC_mm230306_qcd.h5ad')
# caiy2020

caiy2020 = sc.read_h5ad('/Volumes/Lacie/data_lake/Mairi_example/processed_files/abridged_qc/human/Cai2020_scRNA_PBMC_mm230315_qcd.h5ad')
caiy2020

AnnData object with n_obs × n_vars = 73146 × 61533
    obs: 'study', 'individual', 'sample', 'tissue', 'donor', 'age', 'gender', 'status', 'data_type', 'centre', 'version', 'batch', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'percent_mt2', 'n_counts', 'percent_chrY', 'XIST-counts', 'S_score', 'G2M_score', 'predicted_doublets'
    var: 'gene_id', 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    uns: 'donor_colors'
    layers: 'counts', 'sqrt_norm'

- Read in _Cai Y et al 2022_

In [7]:
# caiy2022 = sc.read_h5ad('/lustre/groups/talaveralopez/datasets/tuberculosis/rna-seq/mairi_data_lake/processed_files/abridged_qc/human/Cai2022_scRNA_PBMC_mm230314_qcd.h5ad')
# caiy2022.obs['status'] = 'active_TB'
# caiy2022

caiy2022 = sc.read_h5ad('/Volumes/Lacie/data_lake/Mairi_example/processed_files/abridged_qc/human/Cai2022_scRNA_PBMC_mm230315_qcd.h5ad')
caiy2022.obs['status'] = 'active_TB'
caiy2022

AnnData object with n_obs × n_vars = 25417 × 61533
    obs: 'study', 'individual', 'sample', 'tissue', 'donor', 'data_type', 'centre', 'version', 'object', 'protocol', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'percent_mt2', 'n_counts', 'percent_chrY', 'XIST-counts', 'S_score', 'G2M_score', 'predicted_doublets', 'status'
    var: 'gene_id', 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    uns: 'donor_colors', 'sample_colors'
    layers: 'counts', 'sqrt_norm'

### Merge datasets

In [8]:
caiy_tb = caiy2020.concatenate(caiy2022, batch_key = 'dataset', batch_categories = ['caiy2020', 'caiy2022'], join = 'inner')
caiy_tb

AnnData object with n_obs × n_vars = 98563 × 61533
    obs: 'study', 'individual', 'sample', 'tissue', 'donor', 'age', 'gender', 'status', 'data_type', 'centre', 'version', 'batch', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'percent_mt2', 'n_counts', 'percent_chrY', 'XIST-counts', 'S_score', 'G2M_score', 'predicted_doublets', 'object', 'protocol', 'dataset'
    var: 'gene_id', 'mt', 'ribo', 'n_cells_by_counts-caiy2020', 'mean_counts-caiy2020', 'pct_dropout_by_counts-caiy2020', 'total_counts-caiy2020', 'n_cells_by_counts-caiy2022', 'mean_counts-caiy2022', 'pct_dropout_by_counts-caiy2022', 'total_counts-caiy2022'
    layers: 'counts', 'sqrt_norm'

### Check that anndata object only contains PBMC scRNA from healthy donors

In [9]:
caiy_tb.obs

,study,individual,sample,tissue,donor,age,gender,status,data_type,centre,...,percent_mt2,n_counts,percent_chrY,XIST-counts,S_score,G2M_score,predicted_doublets,object,protocol,dataset
barcode,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGAACAATC-acTB3-caiy2020,CaiY_2021,SAMN14048025,PBMC_TB_3,PBMC,SAMN14048025,34,male,active_TB,scRNAseq,Shenzhen University,...,0.060170,5634.0,0.053248,0.0,-0.352188,-0.193287,0.0,NaN,NaN,caiy2020
AAACCTGAGAAGGTGA-acTB3-caiy2020,CaiY_2021,SAMN14048025,PBMC_TB_3,PBMC,SAMN14048025,34,male,active_TB,scRNAseq,Shenzhen University,...,0.016686,3536.0,0.113122,0.0,-0.064944,-0.071169,0.0,NaN,NaN,caiy2020
AAACCTGAGATCTGCT-acTB3-caiy2020,CaiY_2021,SAMN14048025,PBMC_TB_3,PBMC,SAMN14048025,34,male,active_TB,scRNAseq,Shenzhen University,...,0.099675,1846.0,0.054171,0.0,-0.231399,-0.080643,0.0,NaN,NaN,caiy2020
AAACCTGAGCACAGGT-acTB3-caiy2020,CaiY_2021,SAMN14048025,PBMC_TB_3,PBMC,SAMN14048025,34,male,active_TB,scRNAseq,Shenzhen University,...,0.045580,3993.0,0.075131,0.0,-0.227884,-0.258770,0.0,NaN,NaN,caiy2020
AAACCTGAGCGTGAAC-acTB3-caiy2020,CaiY_2021,SAMN14048025,PBMC_TB_3,PBMC,SAMN14048025,34,male,active_TB,scRNAseq,Shenzhen University,...,0.024570,2035.0,0.147420,0.0,0.020959,0.025030,0.0,NaN,NaN,caiy2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCAGTCCATCCT-HRS100513-pbmc_scRNAseq-caiy2022,CaiY_2022,HRI068293,HRS100513,PBMC,P4,NaN,NaN,active_TB,scRNAseq,Shenzhen University,...,0.052279,746.0,0.000000,0.0,-0.137711,-0.152166,0.0,HRS100513,pbmc_scRNAseq,caiy2022
TTTGTCAGTGCTGTAT-HRS100513-pbmc_scRNAseq-caiy2022,CaiY_2022,HRI068293,HRS100513,PBMC,P4,NaN,NaN,active_TB,scRNAseq,Shenzhen University,...,0.051491,369.0,0.000000,0.0,0.130521,0.092987,0.0,HRS100513,pbmc_scRNAseq,caiy2022
TTTGTCAGTGTGGCTC-HRS100513-pbmc_scRNAseq-caiy2022,CaiY_2022,HRI068293,HRS100513,PBMC,P4,NaN,NaN,active_TB,scRNAseq,Shenzhen University,...,0.028736,348.0,0.000000,0.0,-0.043432,-0.034049,0.0,HRS100513,pbmc_scRNAseq,caiy2022


In [10]:
caiy_tb.obs['data_type'].value_counts()

scRNAseq    98563
Name: data_type, dtype: int64

In [11]:
caiy_tb.obs['tissue'].value_counts()

PBMC    98563
Name: tissue, dtype: int64

In [12]:
caiy_tb.obs['status'].value_counts()

active_TB    58521
Healthy      22049
latent_TB    17993
Name: status, dtype: int64

In [13]:
caiy_healthy = caiy_tb[~caiy_tb.obs['status'].isin(['active_TB', 'latent_TB']),:]

In [15]:
caiy_healthy.obs['status'].value_counts()

Healthy    22049
Name: status, dtype: int64

### Calculate HVGs

In [ ]:
adata = caiy_tb_gex.copy()
adata.layers['counts'] = adata.X.copy()

In [ ]:
sc.pp.highly_variable_genes(
    adata,
    flavor = "seurat_v3",
    n_top_genes = 8000,
    layer = "counts",
    batch_key = "sample",
    subset = True
)

### Data integration with `scVI`

In [ ]:
scvi.model.SCVI.setup_anndata(
    adata,
    layer = "counts",
    categorical_covariate_keys = ["sample"],
    continuous_covariate_keys = ["n_genes", "n_counts"]
)

# Why are the continuous covariate keys as such, and not 'percent mito'/'percent ribo'?


In [ ]:
model = scvi.model.SCVI(adata, n_layers = 3, n_latent = 50, gene_likelihood = "nb", dispersion = 'gene-batch')
model

# Why have all of the parameters not been included?
# What is the "dispersion" argument referring to specifically?


In [ ]:
model.train()

In [ ]:
# Why was the model not saved here?

In [ ]:
latent = model.get_latent_representation()
adata.obsm["X_scVI"] = latent

In [ ]:
sc.pp.neighbors(adata, use_rep = "X_scVI", n_neighbors = 50, metric = 'minkowski')
sc.tl.umap(adata, min_dist = 0.2, spread = 6, random_state = 1712)
sc.pl.umap(adata, frameon = False, color = ['study', 'individual', 'tissue', 'status', 'percent_chrY', 'XIST-counts', 'S_score', 'G2M_score', 'doublet_scores', 'dataset', 'sample'], size = 0.8, legend_fontsize = 5, ncols = 4)

In [ ]:
sc.pl.umap(adata, frameon = False, color = ['CD74', 'CD3E','CD40LG', 'CD8A', 'NKG7', 'CLEC9A', 'CD14', 'FCGR3A'], size = 1, legend_fontsize = 5, legend_loc = 'on data', ncols = 4)

### Use `SCCAF` to select `leiden` resolution

> What is SCCAF? "Single Cell Clustering Assessment Framework"
>> It is a way to cluster cells based on gene expression; it uses repeat applications of clustering and ML models to generate gene expression profiles - it identifies distinct cell groups and a weighted list of feature genes for each group

> What is the architecture/algorithm of the model?
>> ML [logistic regression, random forest, Gaussian process classification, support vector machine and decision tree] and 5-fold CV

In [ ]:
sc.tl.leiden(adata, resolution = 0.7, random_state = 1786)

In [ ]:
import matplotlib.pyplot as plt
from SCCAF import SCCAF_assessment, plot_roc
y_prob, y_pred, y_test, clf, cvsm, acc = SCCAF_assessment(adata.X, adata.obs['leiden'], n = 100)

In [ ]:
plot_roc(y_prob, y_test, clf, cvsm = cvsm, acc = acc)
plt.show()

In [ ]:
sc.pl.umap(adata, frameon = False, color = ['leiden', 'status', 'CD74'], size = 0.8, legend_fontsize = 5, legend_loc = 'on data')

In [ ]:
sc.pl.umap(adata, frameon = False, color = ['leiden', 'status', 'tissue', 'ADH7', 'CDH1', 'CD74', 'CD3E', 'MUC20', 'DUSP4', 'FOXJ1', 'MUC1', 'FOXI1'], size = 1, legend_fontsize = 5)

### Export clustered object

In [ ]:
adata

In [ ]:
caiy_tb_gex

In [ ]:
# Making a hybrid anndata object using sections from both original anndata object and the cai_tb_gex object
adata_export = anndata.AnnData(X = caiy_tb_gex.X, var = caiy_tb_gex.var, obs = adata.obs, uns = adata.uns, obsm = adata.obsm, layers = caiy_tb_gex.layers, obsp = adata.obsp)
adata_export